In [1]:
import glob
import pandas as pd
import utils

In [2]:
d = utils.read_summary("../artifacts-server/mpg-server/cross-validation/mnist-M1/mlp-vib-diag-mnist-cv5-0--2020-01-24--16-07-43-211365/summary.yml")

In [3]:
def read_experiment_summaries(path):
    experiments = glob.glob(f"{path}/summary.yml")
    results = []
    for f in experiments:
        r = utils.read_summary(f)
        r["metrics:test:error_L12"] = 1 - r["metrics:test:accuracy_L12"]
        results.append(r)

    df = pd.DataFrame(results)
    print(f"Reading {path}")
    print(f"we have {len(df)} rows")
    return df

In [7]:
mnist_cv = "../artifacts-server/mpg-server/cross-validation/mnist-M1/*"
df_mnist = read_experiment_summaries(mnist_cv)

Reading ../artifacts-server/mpg-server/cross-validation/mnist-M1/*
we have 250 rows


In [8]:
df_mnist[:2]

,M,batch_size,beta,class_loss,cov_type,dataset,elapsed_time,epoch,lr,lr_schedule,...,metrics:test:accuracy_L1,metrics:test:accuracy_L12,metrics:test:loss,metrics:train:I_XZ,metrics:train:I_YZ,metrics:train:accuracy_L1,metrics:train:accuracy_L12,metrics:train:loss,filename,metrics:test:error_L12
0,1,100,0.0010,vib,diag,mnist-cv5-1,18.806219,200,0.0001,alemi_vib_mnist,...,0.980750,0.987084,0.137123,23.626949,3.317983,0.999562,1.0,0.027572,mlp-vib-diag-mnist-cv5-1--2020-01-24--16-07-43...,0.012916
1,1,100,0.0001,vib,diag,mnist-cv5-0,12.708673,200,0.0001,alemi_vib_mnist,...,0.984167,0.986250,0.098643,43.505867,3.321230,1.000000,1.0,0.005049,mlp-vib-diag-mnist-cv5-0--2020-01-24--16-50-52...,0.013750


In [9]:
def print_stats(df):
    df_temp = df.groupby(["strategy", "beta"])\
        .agg(
            mean_acc=("metrics:test:error_L12", "mean"),
            std_acc=("metrics:test:error_L12", "std"),
            count=("metrics:test:error_L12", "count"),
        )\
        .reset_index() 

    print(df_temp)


    df_temp['metric'] = df_temp[["mean_acc", "std_acc"]].apply(lambda x: "%.4f±%.4f" % (x["mean_acc"], x["std_acc"]), axis=1)

    df_temp = df_temp.pivot(index='strategy', columns='beta', values='metric')
    print("--------")
    print(df_temp.to_csv())
    
print_stats(df_mnist)

      strategy      beta  mean_acc   std_acc  count
0   algo2/k:10  0.000001  0.014983  0.001612      5
1   algo2/k:10  0.000005  0.014683  0.001746      5
2   algo2/k:10  0.000008  0.014200  0.001632      5
3   algo2/k:10  0.000010  0.013933  0.001931      5
4   algo2/k:10  0.000050  0.014166  0.001342      5
5   algo2/k:10  0.000080  0.013900  0.001621      5
6   algo2/k:10  0.000100  0.014316  0.001697      5
7   algo2/k:10  0.000500  0.014333  0.001774      5
8   algo2/k:10  0.000800  0.014183  0.002108      5
9   algo2/k:10  0.001000  0.014517  0.001495      5
10  algo2/k:20  0.000001  0.015266  0.001818      5
11  algo2/k:20  0.000005  0.014316  0.001502      5
12  algo2/k:20  0.000008  0.013950  0.001655      5
13  algo2/k:20  0.000010  0.014316  0.001452      5
14  algo2/k:20  0.000050  0.013716  0.002422      5
15  algo2/k:20  0.000080  0.013467  0.001497      5
16  algo2/k:20  0.000100  0.013333  0.001975      5
17  algo2/k:20  0.000500  0.013983  0.001787      5
18  algo2/k:

In [14]:
cifar10_cv = "../artifacts-server/mpg-server/cross-validation/cifar10-M1/*"
df_cifar10 = read_experiment_summaries(cifar10_cv)

Reading ../artifacts-server/mpg-server/cross-validation/cifar10-M1/*
we have 100 rows


In [15]:
print_stats(df_cifar10)

      strategy     beta  mean_acc   std_acc  count
0   algo2/k:10  0.00000  0.099022  0.003273      5
1   algo2/k:10  0.00001  0.100020  0.003442      5
2   algo2/k:10  0.00010  0.097684  0.003338      5
3   algo2/k:10  0.00100  0.098003  0.002954      5
4   algo2/k:20  0.00000  0.098722  0.003869      5
5   algo2/k:20  0.00001  0.097224  0.003638      5
6   algo2/k:20  0.00010  0.099301  0.004535      5
7   algo2/k:20  0.00100  0.098462  0.003282      5
8   algo2/k:30  0.00000  0.098802  0.002845      5
9   algo2/k:30  0.00001  0.099321  0.003561      5
10  algo2/k:30  0.00010  0.101038  0.004721      5
11  algo2/k:30  0.00100  0.099780  0.005120      5
12   algo2/k:5  0.00000  0.099022  0.002838      5
13   algo2/k:5  0.00001  0.100919  0.004022      5
14   algo2/k:5  0.00010  0.101338  0.003321      5
15   algo2/k:5  0.00100  0.098842  0.005409      5
16     oneshot  0.00000  0.100339  0.003921      5
17     oneshot  0.00001  0.097843  0.001876      5
18     oneshot  0.00010  0.0981

In [16]:
df_cifar10[ (df_cifar10.beta == 0) & (df_cifar10.strategy == "oneshot") ]

,M,batch_size,beta,class_loss,cov_type,data_augmentation,dataset,elapsed_time,epoch,lr,...,metrics:test:accuracy_L1,metrics:test:accuracy_L12,metrics:test:loss,metrics:train:I_XZ,metrics:train:I_YZ,metrics:train:accuracy_L1,metrics:train:accuracy_L12,metrics:train:loss,filename,metrics:test:error_L12
5,1,32,0.0,vib,diag,True,cifar10-cv5-3,149.392475,200,0.0001,...,0.898163,0.898063,0.715558,458.363312,3.278093,0.989950,0.990050,0.043837,resnet20-vib-diag-cifar10-cv5-3--2020-01-31--0...,0.101937
20,1,32,0.0,vib,diag,True,cifar10-cv5-4,151.696281,200,0.0001,...,0.903255,0.903255,0.643713,451.179047,3.274782,0.989000,0.989025,0.047144,resnet20-vib-diag-cifar10-cv5-4--2020-01-31--0...,0.096745
23,1,32,0.0,vib,diag,True,cifar10-cv5-1,156.417425,200,0.0001,...,0.896166,0.896166,0.695349,500.075287,3.272556,0.988525,0.988525,0.049375,resnet20-vib-diag-cifar10-cv5-1--2020-02-04--2...,0.103834
39,1,32,0.0,vib,diag,True,cifar10-cv5-0,151.985752,200,0.0001,...,0.904453,0.904453,0.634488,434.292297,3.274707,0.988775,0.988775,0.047221,resnet20-vib-diag-cifar10-cv5-0--2020-02-04--2...,0.095547
85,1,32,0.0,vib,diag,True,cifar10-cv5-2,149.818658,200,0.0001,...,0.896565,0.896366,0.693999,462.863403,3.270240,0.987950,0.987950,0.051689,resnet20-vib-diag-cifar10-cv5-2--2020-01-31--0...,0.103634
